# Neural Network Regression task - Bike sharing

Bike sharing systems are new generation of traditional bike rentals where whole process from membership, rental and return has become automatic. Through these systems, a user is able to easily rent a bike from a particular position and return it at another place.

The dataset contains the hourly count of rental bikes between years 2011 and 2012 in the Capital Bikeshare system (Wasington DC) with the corresponding weather and seasonal information.

The goal of this task is to train a regressor to predict total counts of bike rentals based on the provided features for a given hour. 

## Data source
[http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset)

## Feature description
* **dteday** - date time stamot
* **season** - season (1: spring, 2: summer, 3: fall, 4: winter)
* **yr** - year (0: 2011, 1: 2012)
* **mnth** - month (1 to 12)
* **hr** - hour (0 to 23)
* **holiday** - 1 if the day is a holiday, else 0 (extracted from [holiday schedules](https://dchr.dc.gov/page/holiday-schedules))
* **weekday** - day of the week (0 to 6)
* **workingday** - is 1 if day is neither weekend nor holiday, else 0.
* **weathersit** 
    * 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    * 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    * 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    * 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
* **temp** - Normalized temperature in degrees of Celsius.
* **atemp** - Normalized feeling temperature in degrees Celsius.
* **hum** - Normalized relative humidity.
* **windspeed** - Normalized wind speed.
* **casual** - Count of casual users.
* **registered** - Count of registered users.
* **cnt** -  Count of total rental bikes including both casual and registered. This is the target value. 

In [1]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/mlcollege/introduction-to-ml/master/data/bikes.csv', sep=',')
data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Add some features from the past

Since we have time stamp of every measurement, we can see the data as a time series and use data from the past. We add one feature column computed from the data of the previous hour.

In [2]:
data.sort_values(['dteday', 'hr'])
cnt = data['cnt']
data['hist'] = cnt.shift(1)
data = data[1:]
data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,hist
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40,16.0
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32,40.0
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13,32.0
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1,13.0
5,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1,1.0


## Neural Network regressor

Implement a neural network regressor based on all reasonable features from the input data set. Notice that some of the features from the input data cannot be used.

### Data preparation

Prepare train and test data sets.

In [3]:
from sklearn.model_selection import train_test_split

X_all = data[['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit','temp', 'atemp', 'hum', 'windspeed', 'hist']]
y_all = data['cnt']


X_train, X_test, y_train, y_test = train_test_split(
    X_all, 
    y_all,
    random_state=1,
    test_size=0.2)

print('Train size: {}'.format(len(X_train)))
print('Test size: {}'.format(len(X_test)))

Train size: 13902
Test size: 3476


Standardize the features

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Training a regressor
Design and train a regression model. Use the [mean squared error](https://keras.io/losses/) loss function. Experiment with various architectures, [activation functions](https://keras.io/activations/) and [optimizers](https://keras.io/optimizers/).

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential()

model.add(Dense(32, input_shape=(13, )))
model.add(Activation('tanh'))
#model.add(Dropout(0.5))
model.add(Dense(32))
model.add(Activation('tanh'))
#model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('relu'))

Compile the model

In [9]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae'])

Train the model

In [10]:
model.fit(X_train, y_train,
          batch_size = 128, epochs = 500, verbose=1,
          validation_data=(X_test, y_test))

Train on 13902 samples, validate on 3476 samples
Epoch 1/500
13902/13902 [==============================] - 0s 24us/step - loss: 67412.2591 - mean_absolute_error: 187.1490 - val_loss: 66408.0754 - val_mean_absolute_error: 185.5463
Epoch 2/500
13902/13902 [==============================] - 0s 11us/step - loss: 65134.7504 - mean_absolute_error: 183.0098 - val_loss: 64584.7864 - val_mean_absolute_error: 181.9535
Epoch 3/500
13902/13902 [==============================] - 0s 11us/step - loss: 63310.9110 - mean_absolute_error: 179.0131 - val_loss: 62451.0544 - val_mean_absolute_error: 176.2188
Epoch 4/500
13902/13902 [==============================] - 0s 12us/step - loss: 61152.4471 - mean_absolute_error: 172.5322 - val_loss: 60698.0906 - val_mean_absolute_error: 171.6683
Epoch 5/500
13902/13902 [==============================] - 0s 11us/step - loss: 59702.8073 - mean_absolute_error: 169.6402 - val_loss: 59359.8162 - val_mean_absolute_error: 169.1705
Epoch 6/500
13902/13902 [================

Epoch 45/500
13902/13902 [==============================] - 0s 12us/step - loss: 28109.3223 - mean_absolute_error: 94.0084 - val_loss: 27917.6900 - val_mean_absolute_error: 94.1245
Epoch 46/500
13902/13902 [==============================] - 0s 11us/step - loss: 27613.1031 - mean_absolute_error: 92.9139 - val_loss: 27417.1998 - val_mean_absolute_error: 92.9516
Epoch 47/500
13902/13902 [==============================] - 0s 11us/step - loss: 27123.5048 - mean_absolute_error: 91.6649 - val_loss: 26934.5618 - val_mean_absolute_error: 91.7922
Epoch 48/500
13902/13902 [==============================] - 0s 12us/step - loss: 26647.4440 - mean_absolute_error: 90.5304 - val_loss: 26452.6352 - val_mean_absolute_error: 90.7018
Epoch 49/500
13902/13902 [==============================] - 0s 16us/step - loss: 26178.0131 - mean_absolute_error: 89.3874 - val_loss: 25984.3960 - val_mean_absolute_error: 89.6004
Epoch 50/500
13902/13902 [==============================] - 0s 17us/step - loss: 25719.8718 - m

13902/13902 [==============================] - 0s 14us/step - loss: 5480.4513 - mean_absolute_error: 36.0780 - val_loss: 5284.5175 - val_mean_absolute_error: 36.2918
Epoch 136/500
13902/13902 [==============================] - 0s 17us/step - loss: 5383.5896 - mean_absolute_error: 35.8534 - val_loss: 5196.8612 - val_mean_absolute_error: 36.2333
Epoch 137/500
13902/13902 [==============================] - 0s 18us/step - loss: 5283.6350 - mean_absolute_error: 35.4947 - val_loss: 5109.0102 - val_mean_absolute_error: 35.7933
Epoch 138/500
13902/13902 [==============================] - 0s 17us/step - loss: 5192.3972 - mean_absolute_error: 35.2382 - val_loss: 5005.8750 - val_mean_absolute_error: 35.4984
Epoch 139/500
13902/13902 [==============================] - 0s 18us/step - loss: 5092.9059 - mean_absolute_error: 34.8636 - val_loss: 4930.8178 - val_mean_absolute_error: 35.1559
Epoch 140/500
13902/13902 [==============================] - 0s 12us/step - loss: 5003.1945 - mean_absolute_error:

13902/13902 [==============================] - 0s 11us/step - loss: 1467.4504 - mean_absolute_error: 22.2691 - val_loss: 1485.8141 - val_mean_absolute_error: 22.9643
Epoch 226/500
13902/13902 [==============================] - 0s 10us/step - loss: 1462.1931 - mean_absolute_error: 22.2295 - val_loss: 1469.6486 - val_mean_absolute_error: 22.6986
Epoch 227/500
13902/13902 [==============================] - 0s 10us/step - loss: 1446.3455 - mean_absolute_error: 22.2120 - val_loss: 1473.5974 - val_mean_absolute_error: 22.7853
Epoch 228/500
13902/13902 [==============================] - 0s 12us/step - loss: 1429.1369 - mean_absolute_error: 22.0899 - val_loss: 1478.5279 - val_mean_absolute_error: 22.8272
Epoch 229/500
13902/13902 [==============================] - 0s 12us/step - loss: 1422.4736 - mean_absolute_error: 22.0948 - val_loss: 1451.1880 - val_mean_absolute_error: 22.7608
Epoch 230/500
13902/13902 [==============================] - 0s 12us/step - loss: 1413.3310 - mean_absolute_error:

13902/13902 [==============================] - 0s 11us/step - loss: 991.9955 - mean_absolute_error: 20.0118 - val_loss: 1121.8862 - val_mean_absolute_error: 21.0429
Epoch 316/500
13902/13902 [==============================] - 0s 10us/step - loss: 991.9074 - mean_absolute_error: 19.9699 - val_loss: 1102.4848 - val_mean_absolute_error: 20.9100
Epoch 317/500
13902/13902 [==============================] - 0s 12us/step - loss: 987.0594 - mean_absolute_error: 19.9565 - val_loss: 1123.0225 - val_mean_absolute_error: 21.1389
Epoch 318/500
13902/13902 [==============================] - 0s 13us/step - loss: 984.2534 - mean_absolute_error: 19.9507 - val_loss: 1110.5906 - val_mean_absolute_error: 20.9653
Epoch 319/500
13902/13902 [==============================] - 0s 12us/step - loss: 986.0818 - mean_absolute_error: 19.8928 - val_loss: 1098.0179 - val_mean_absolute_error: 20.9389
Epoch 320/500
13902/13902 [==============================] - 0s 13us/step - loss: 986.3544 - mean_absolute_error: 19.99

Epoch 361/500
13902/13902 [==============================] - 0s 10us/step - loss: 924.4150 - mean_absolute_error: 19.4603 - val_loss: 1071.4204 - val_mean_absolute_error: 20.6363
Epoch 362/500
13902/13902 [==============================] - 0s 10us/step - loss: 925.8529 - mean_absolute_error: 19.4796 - val_loss: 1065.8262 - val_mean_absolute_error: 20.6998
Epoch 363/500
13902/13902 [==============================] - 0s 9us/step - loss: 918.9915 - mean_absolute_error: 19.4406 - val_loss: 1066.1851 - val_mean_absolute_error: 20.6526
Epoch 364/500
13902/13902 [==============================] - 0s 11us/step - loss: 921.6053 - mean_absolute_error: 19.4845 - val_loss: 1068.3524 - val_mean_absolute_error: 20.7490
Epoch 365/500
13902/13902 [==============================] - 0s 10us/step - loss: 912.9242 - mean_absolute_error: 19.3991 - val_loss: 1052.7603 - val_mean_absolute_error: 20.4194
Epoch 366/500
13902/13902 [==============================] - 0s 10us/step - loss: 907.4520 - mean_absolute

Epoch 407/500
13902/13902 [==============================] - 0s 12us/step - loss: 871.0840 - mean_absolute_error: 19.0639 - val_loss: 1032.8284 - val_mean_absolute_error: 20.3966
Epoch 408/500
13902/13902 [==============================] - 0s 11us/step - loss: 870.8325 - mean_absolute_error: 19.0326 - val_loss: 1024.8385 - val_mean_absolute_error: 20.2143
Epoch 409/500
13902/13902 [==============================] - 0s 11us/step - loss: 869.3702 - mean_absolute_error: 19.0340 - val_loss: 1039.2132 - val_mean_absolute_error: 20.3316
Epoch 410/500
13902/13902 [==============================] - 0s 10us/step - loss: 870.4460 - mean_absolute_error: 19.0495 - val_loss: 1058.0069 - val_mean_absolute_error: 20.3127
Epoch 411/500
13902/13902 [==============================] - 0s 10us/step - loss: 867.8957 - mean_absolute_error: 19.0288 - val_loss: 1045.8109 - val_mean_absolute_error: 20.4508
Epoch 412/500
13902/13902 [==============================] - 0s 10us/step - loss: 865.2492 - mean_absolut

Epoch 453/500
13902/13902 [==============================] - 0s 17us/step - loss: 838.6948 - mean_absolute_error: 18.7571 - val_loss: 1027.7998 - val_mean_absolute_error: 20.3360
Epoch 454/500
13902/13902 [==============================] - 0s 15us/step - loss: 841.0417 - mean_absolute_error: 18.7730 - val_loss: 1026.9598 - val_mean_absolute_error: 20.3168
Epoch 455/500
13902/13902 [==============================] - 0s 17us/step - loss: 837.4463 - mean_absolute_error: 18.7226 - val_loss: 1032.7257 - val_mean_absolute_error: 20.3014
Epoch 456/500
13902/13902 [==============================] - 0s 18us/step - loss: 833.3170 - mean_absolute_error: 18.6588 - val_loss: 1022.8011 - val_mean_absolute_error: 20.1426
Epoch 457/500
13902/13902 [==============================] - 0s 17us/step - loss: 842.5921 - mean_absolute_error: 18.7773 - val_loss: 1024.6099 - val_mean_absolute_error: 20.1793
Epoch 458/500
13902/13902 [==============================] - 0s 18us/step - loss: 835.7005 - mean_absolut

### Evaluate the models

Measure mean squared error and mean absolute error evaluation metrics on both train and test data sets. Compute the mean and standard deviation of the target values.

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

y_pred = model.predict(X_test)
print ("Test mean: {}, std: {}".format(np.mean(y_test), np.std(y_test)))
print("Test Root mean squared error: {:.2f}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
print("Test Mean absolute error: {:.2f}".format(mean_absolute_error(y_test, y_pred)))

Test mean: 190.1343498273878, std: 181.50427424823098
Test Root mean squared error: 32.25
Test Mean absolute error: 20.09


In [12]:
y_pred = model.predict(X_train)
print("Train Root mean squared error: %.2f"
      % np.sqrt(mean_squared_error(y_train, y_pred)))
print("Train Mean absolute error: %.2f"
      % mean_absolute_error(y_train, y_pred))

Train Root mean squared error: 28.54
Train Mean absolute error: 18.42
